In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

In [38]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key="")

In [39]:
PROMPT_TEMPLATE = """
You are looking to watch a movie of genre {genre}, this weekend, in order to pick 
what movie to watch you need to propose the options with the following data:
- Title
- Genre/subgenre
- User rating
- Critics rating
- Release year
- duration minutes
- gore level

Generate {n_examples} recommendations using the {format_instructions}
"""

In [40]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
import datetime

class Movie(BaseModel):
    title: str
    genre_subgenre: str
    user_rating: float
    critics_rating: float
    release_year: int
    duration_minutes: int
    gore_level: float

class MovieCommends(BaseModel):
    genre: str
    recommendations: list[Movie]

out_parser = PydanticOutputParser(pydantic_object=MovieCommends)

prompt_tpl = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["genre", "n_examples"],
    partial_variables=
    {
        "format_instructions": out_parser.get_format_instructions()
    },
)

In [41]:
class MoviesParams(BaseModel):
    genre: str
    n_examples: int

params = MoviesParams(genre="romantic", n_examples=4)

llm_input = prompt_tpl.format(**params.dict())
prediction = llm.predict(llm_input)

In [42]:
movies = out_parser.parse(prediction)

In [43]:
movies.genre

'romantic'

In [44]:
for m in movies.recommendations:
    print(m.title, m.gore_level, m.user_rating, m.release_year)

The Notebook 0.0 8.1 2004
La La Land 0.0 8.0 2016
Eternal Sunshine of the Spotless Mind 0.0 8.3 2004
Crazy, Stupid, Love. 0.0 7.4 2011
